In [1]:
from datetime import datetime
from ctypes import cdll,c_long, c_ulong, c_uint32,byref,create_string_buffer,c_bool,c_char_p,c_int,c_int16,c_double, sizeof, c_voidp
from TLPMX import TLPMX
import time

from TLPMX import TLPM_DEFAULT_CHANNEL

c:\Users\kaufmanlab\Desktop\power meter\TLPMX.py:292: SyntaxWarning: invalid escape sequence '\P'
  dllabspath = "C:\Program Files\IVI Foundation\VISA\Win64\Bin\TLPMX_64.dll"
c:\Users\kaufmanlab\Desktop\power meter\TLPMX.py:297: SyntaxWarning: invalid escape sequence '\P'
  dllabspath = "C:\Program Files\IVI Foundation\VISA\Win64\Bin\TLPMX_64.dll"


In [6]:
# Find connected power meter devices.
tlPM = TLPMX()
deviceCount = c_uint32()
tlPM.findRsrc(byref(deviceCount))

print("Number of found devices: " + str(deviceCount.value))
print("")

resourceName = create_string_buffer(1024)

for i in range(0, deviceCount.value):
    tlPM.getRsrcName(c_int(i), resourceName)
    print("Resource name of device", i, ":", c_char_p(resourceName.raw).value)
print("")
err = tlPM.close()

Number of found devices: 1

Resource name of device 0 : b'USB0::0x1313::0x8078::P0045353::INSTR'



In [7]:
# Connect to last device.
tlPM = TLPMX()
tlPM.open(resourceName, c_bool(True), c_bool(True))

message = create_string_buffer(1024)
tlPM.getCalibrationMsg(message,TLPM_DEFAULT_CHANNEL)
print("Connected to device", i)
print("Last calibration date: ",c_char_p(message.raw).value)
print("")

time.sleep(2)

# Set wavelength in nm.
wavelength = c_double(671)
tlPM.setWavelength(wavelength,TLPM_DEFAULT_CHANNEL)

# Enable auto-range mode.
# 0 -> auto-range disabled
# 1 -> auto-range enabled
tlPM.setPowerAutoRange(c_int16(1),TLPM_DEFAULT_CHANNEL)

# Set power unit to Watt.
# 0 -> Watt
# 1 -> dBm
tlPM.setPowerUnit(c_int16(0),TLPM_DEFAULT_CHANNEL)

# Take power measurements and save results to arrays.
power_measurements = []
times = []
count = 0
while count < 5:
    power =  c_double()
    tlPM.measPower(byref(power),TLPM_DEFAULT_CHANNEL)
    power_measurements.append(power.value)
    times.append(datetime.now())
    print(times[count], ":", power_measurements[count], "W")
    count+=1
    time.sleep(1)
print("")

# Close power meter connection.
tlPM.close()
print('End program')


Connected to device 0
Last calibration date:  b'13-Feb-2024'

2024-11-13 21:48:44.829347 : 3.64508924e-05 W
2024-11-13 21:48:45.838268 : 3.65711421e-05 W
2024-11-13 21:48:46.845826 : 3.64508924e-05 W
2024-11-13 21:48:47.863400 : 3.65735032e-05 W
2024-11-13 21:48:48.878422 : 3.65192718e-05 W

End program
